# This is an INFERENCE using the model combine the method of MLP, CNN and Transformer

Thanks PICEKL for his baseline, which help me a lot.

The main idea of this method is to obtain multiple features from multiple dimensions. We simply divide it into four types: **image information** with spatial characteristics, **meta information** that is independent of each other, **climate information** and **satellite information** with time characteristics.

In [1]:
import os
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, repeat
from pathlib import Path
import time

In [2]:
new_dir = Path("output") / time.strftime('%Y-%m-%d_%H%M', time.localtime())
new_dir.mkdir(parents=True, exist_ok=True)

In [3]:
num_classes = 11255
num_epochs = 100
seed = 113

When reading data, different **fusion** and **normalization** will be performed for different types of data.

In [4]:
torch.manual_seed(seed)

In [5]:
class TrainDataset(Dataset):
    def __init__(self, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.metadata = metadata

        labels = self.metadata[['surveyId' ,'speciesId']].astype(int).copy()
        self.label_dict = labels.groupby('surveyId')['speciesId'].apply(list).to_dict()

        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)
        self.metadata.fillna(0,inplace=True)
        self.metadata.replace({float('-inf'): 0}, inplace=True)
        self.metadata_data = self.Norm(self.metadata.iloc[:,:5])

        self.merge_key = 'surveyId'
        self.climate_average = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Average 1981-2010/GLC24-PA-train-bioclimatic.csv")
        self.climate_monthly = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Monthly/GLC24-PA-train-bioclimatic_monthly.csv")
        self.climate = pd.merge(self.climate_average, self.climate_monthly, on=self.merge_key)
        self.climate.fillna(self.climate.mean(),inplace=True)
        self.climate_data = self.Norm_all(self.climate)

        self.landsat_b = pd.read_csv("data/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-blue.csv")
        self.landsat_b.fillna(self.landsat_b.mean(),inplace=True)
        self.landsat_g = pd.read_csv("data/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-green.csv")
        self.landsat_g.fillna(self.landsat_g.mean(),inplace=True)
        self.landsat_r = pd.read_csv("data/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-red.csv")
        self.landsat_r.fillna(self.landsat_r.mean(),inplace=True)
        self.landsat_n = pd.read_csv("data/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-nir.csv")
        self.landsat_n.fillna(self.landsat_n.mean(),inplace=True)
        self.landsat_s1 = pd.read_csv("data/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-swir1.csv")
        self.landsat_s1.fillna(self.landsat_s1.mean(),inplace=True)
        self.landsat_s2 = pd.read_csv("data/geolifeclef-2024/PA-train-landsat_time_series/GLC24-PA-train-landsat_time_series-swir2.csv")
        self.landsat_s2.fillna(self.landsat_s2.mean(),inplace=True)
        self.landsat_data = torch.cat([self.Norm_all(self.landsat_b),self.Norm_all(self.landsat_g),self.Norm_all(self.landsat_r),self.Norm_all(self.landsat_n),self.Norm_all(self.landsat_s1),self.Norm_all(self.landsat_s2)],axis=1)

        self.elevation = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Elevation/GLC24-PA-train-elevation.csv")
        self.elevation[self.elevation<0]=0
        self.elevation.fillna(self.elevation.mean(),inplace=True)
        self.elevation_data = self.Norm(self.elevation)

        self.human_footprint = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Human Footprint/GLC24-PA-train-human_footprint.csv")
        self.human_footprint[self.human_footprint<0]=0
        self.human_footprint.fillna(self.human_footprint.mean(),inplace=True)
        self.human_footprint_data = self.Norm(self.human_footprint)

        self.landcover = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/LandCover/GLC24-PA-train-landcover.csv")
        self.landcover[self.landcover<0]=0
        self.landcover.fillna(self.landcover.mean(),inplace=True)
        self.landcover_data = self.Norm(self.landcover)

        self.soilgrids = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/SoilGrids/GLC24-PA-train-soilgrids.csv")
        self.soilgrids[self.soilgrids<0]=0
        self.soilgrids.fillna(self.soilgrids.mean(),inplace=True)
        self.soilgrids_data = self.Norm(self.soilgrids)

        self.metadata_data = torch.cat((self.metadata_data, self.elevation_data, self.human_footprint_data, self.landcover_data, self.soilgrids_data), dim=1)

    def Norm(self,df):
        output=torch.from_numpy(df.iloc[:,1:].values).float()
        return (output-output.mean(dim=0))/output.std(dim=0)

    def Norm_all(self,df):
        output=torch.from_numpy(df.iloc[:,1:].values).float()
        return (output-output.mean())/output.std()

    def patch_rgb_path(self,survey_id):
        path = "data/geolifeclef-2024/PA_Train_SatellitePatches_RGB/pa_train_patches_rgb"
        for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
            path = os.path.join(path, d)
        path = os.path.join(path, f"{survey_id}.jpeg")
        return path

    def patch_nir_path(self,survey_id):
        path = "data/geolifeclef-2024/PA_Train_SatellitePatches_NIR/pa_train_patches_nir"
        for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
            path = os.path.join(path, d)
        path = os.path.join(path, f"{survey_id}.jpeg")
        return path

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):

        survey_id = self.metadata.surveyId[idx]

        image_path = self.patch_rgb_path(survey_id)
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)
        image = image.unsqueeze(0)
        image_nir_path = self.patch_nir_path(survey_id)
        nir_image = Image.open(image_nir_path).convert("L")
        nir_image = self.transform(nir_image)
        nir_image = nir_image.unsqueeze(0)
        image_data = torch.cat([image,nir_image],dim=1)
        image_data = torch.squeeze(image_data)
        sample=[self.metadata_data[idx,:],image_data,self.landsat_data[idx,:],self.climate_data[idx,:]]
        species_ids = self.label_dict[survey_id]  # Get list of species IDs for the survey ID
        label = torch.zeros(num_classes)  # Initialize label tensor
        for species_id in species_ids:
            label[species_id] = 1  # Set the corresponding class index to 1 for each species ID
        count = len(species_ids)
        return sample, survey_id, label, count#, species_ids

In [6]:
class TestDataset(Dataset):
    def __init__(self, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.metadata = metadata

        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)
        self.metadata.fillna(0,inplace=True)
        self.metadata.replace({float('-inf'): 0}, inplace=True)
        self.metadata_data = self.Norm(self.metadata.iloc[:,:5])

        self.merge_key = 'surveyId'
        self.climate_average = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Average 1981-2010/GLC24-PA-test-bioclimatic.csv")
        self.climate_monthly = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Climate/Monthly/GLC24-PA-test-bioclimatic_monthly.csv")
        self.climate = pd.merge(self.climate_average, self.climate_monthly, on=self.merge_key)
        self.climate.fillna(self.climate.mean(),inplace=True)
        self.climate_data = self.Norm_all(self.climate)

        self.landsat_b = pd.read_csv("data/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-blue.csv")
        self.landsat_b.fillna(self.landsat_b.mean(),inplace=True)
        self.landsat_g = pd.read_csv("data/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-green.csv")
        self.landsat_g.fillna(self.landsat_g.mean(),inplace=True)
        self.landsat_r = pd.read_csv("data/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-red.csv")
        self.landsat_r.fillna(self.landsat_r.mean(),inplace=True)
        self.landsat_n = pd.read_csv("data/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-nir.csv")
        self.landsat_n.fillna(self.landsat_n.mean(),inplace=True)
        self.landsat_s1 = pd.read_csv("data/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-swir1.csv")
        self.landsat_s1.fillna(self.landsat_s1.mean(),inplace=True)
        self.landsat_s2 = pd.read_csv("data/geolifeclef-2024/PA-test-landsat_time_series/GLC24-PA-test-landsat_time_series-swir2.csv")
        self.landsat_s2.fillna(self.landsat_s2.mean(),inplace=True)
        self.landsat_data = torch.cat([self.Norm_all(self.landsat_b),self.Norm_all(self.landsat_g),self.Norm_all(self.landsat_r),self.Norm_all(self.landsat_n),self.Norm_all(self.landsat_s1),self.Norm_all(self.landsat_s2)],axis=1)

        self.elevation = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Elevation/GLC24-PA-test-elevation.csv")
        self.elevation[self.elevation<0]=0
        self.elevation.fillna(self.elevation.mean(),inplace=True)
        self.elevation_data = self.Norm(self.elevation)

        self.human_footprint = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/Human Footprint/GLC24-PA-test-human_footprint.csv")
        self.human_footprint[self.human_footprint<0]=0
        self.human_footprint.fillna(self.human_footprint.mean(),inplace=True)
        self.human_footprint_data = self.Norm(self.human_footprint)

        self.landcover = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/LandCover/GLC24-PA-test-landcover.csv")
        self.landcover[self.landcover<0]=0
        self.landcover.fillna(self.landcover.mean(),inplace=True)
        self.landcover_data = self.Norm(self.landcover)

        self.soilgrids = pd.read_csv("data/geolifeclef-2024/EnvironmentalRasters/EnvironmentalRasters/SoilGrids/GLC24-PA-test-soilgrids.csv")
        self.soilgrids[self.soilgrids<0]=0
        self.soilgrids.fillna(self.soilgrids.mean(),inplace=True)
        self.soilgrids_data = self.Norm(self.soilgrids)

        self.metadata_data = torch.cat((self.metadata_data, self.elevation_data, self.human_footprint_data, self.landcover_data, self.soilgrids_data), dim=1)

    def Norm(self,df):
        output=torch.from_numpy(df.iloc[:,1:].values).float()
        return (output-output.mean(dim=0))/output.std(dim=0)

    def Norm_all(self,df):
        output=torch.from_numpy(df.iloc[:,1:].values).float()
        return (output-output.mean())/output.std()

    def patch_rgb_path(self,survey_id):
        path = "data/geolifeclef-2024/PA_Test_SatellitePatches_RGB/pa_test_patches_rgb"
        for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
            path = os.path.join(path, d)
        path = os.path.join(path, f"{survey_id}.jpeg")
        return path

    def patch_nir_path(self,survey_id):
        path = "data/geolifeclef-2024/PA_Test_SatellitePatches_NIR/pa_test_patches_nir"
        for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
            path = os.path.join(path, d)
        path = os.path.join(path, f"{survey_id}.jpeg")
        return path

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):

        survey_id = self.metadata.surveyId[idx]

        image_path = self.patch_rgb_path(survey_id)
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)
        image = image.unsqueeze(0)
        image_nir_path = self.patch_nir_path(survey_id)
        nir_image = Image.open(image_nir_path).convert("L")
        nir_image = self.transform(nir_image)
        nir_image = nir_image.unsqueeze(0)
        image_data = torch.cat([image,nir_image],dim=1)
        image_data = torch.squeeze(image_data)
        sample=[self.metadata_data[idx,:],image_data,self.landsat_data[idx,:],self.climate_data[idx,:]]
        return sample, survey_id

In [7]:
# Dataset and DataLoader
train_batch_size = 64
test_batch_size = 1

transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

# Load training data
train_metadata_path = "data/geolifeclef-2024/GLC24_PA_metadata_train.csv"
train_metadata = pd.read_csv(train_metadata_path)
train_dataset = TrainDataset(train_metadata, subset="train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=False, num_workers=1)

# Load testing data
test_metadata_path = "data/geolifeclef-2024/GLC24_PA_metadata_test.csv"
test_metadata = pd.read_csv(test_metadata_path)
test_dataset = TestDataset(test_metadata, subset="test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=1)

This is an MLP used to extract features from generally independent information.

In [8]:
class Embedding(nn.Module):
    def __init__(self, dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, out_dim*5)
        self.fc2 = nn.Linear(out_dim*5, out_dim)
        self.norm = nn.LayerNorm(out_dim*5)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = self.norm(x)
        x = self.fc2(x)
        return x

The following is the part of ViT, which can also be considered as the Encoder part of **Transformer**.

In [9]:
class Multihead_self_attention(nn.Module):
    def __init__(self, heads, head_dim, dim):
        super().__init__()
        self.head_dim = head_dim
        self.heads = heads
        self.inner_dim = self.heads*self.head_dim
        self.scale = self.head_dim**-0.5
        self.to_qkv = nn.Linear(dim, self.inner_dim*3)
        self.to_output = nn.Linear(self.inner_dim, dim)
        self.norm = nn.LayerNorm(dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        Q, K, V = map(lambda t: rearrange(t, 'b l (h dim) -> b h l dim', dim=self.head_dim), qkv)
        K_T = K.transpose(-1, -2)
        att_score = Q@K_T*self.scale
        att = self.softmax(att_score)
        out = att@V   # (B,H,L,dim)
        out = rearrange(out, 'b h l dim -> b l (h dim)')
        output = self.to_output(out)
        return output

In [10]:
class FeedForward(nn.Module):
    def __init__(self, dim, mlp_dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, mlp_dim)
        self.fc2 = nn.Linear(mlp_dim, dim)
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        x = self.norm(x)
        x = F.gelu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
class Transformer_block(nn.Module):
    def __init__(self, dim, heads, head_dim, mlp_dim):
        super().__init__()
        self.MHA = Multihead_self_attention(heads=heads, head_dim=head_dim, dim=dim)
        self.FeedForward = FeedForward(dim=dim, mlp_dim=mlp_dim)

    def forward(self, x):
        x = self.MHA(x)+x
        x = self.FeedForward(x)+x
        return x

In [12]:
class ViT(nn.Module):
    def __init__(self, dim, heads, head_dim, mlp_dim, num_class):
        super().__init__()
        self.transformer = Transformer_block(dim=dim, heads=heads, head_dim=head_dim, mlp_dim=mlp_dim)

        self.MLP_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_class)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.transformer(x)
        CLS_token = x[:, 0, :]
        out = self.MLP_head(CLS_token)
        return out

The following is **CNN**, used to extract feature information from images.

In [13]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()

        self.resnet18 = models.resnet18(weights=None)
        self.resnet18.conv1 = nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet18.maxpool = nn.Identity()
        self.ln = nn.LayerNorm(1000)
        self.fc1 = nn.Linear(1000, num_classes)

    def forward(self, x):
        x = self.resnet18(x)
        x = self.ln(x)
        x = self.fc1(x)
        return x

This is the **final multi-modal model**. Unlike ViT, its input is features extracted from each dimension. 

For features with time information, I added additional position information. 

At the same time, I also added a feature that is a fusion of the first four features.

In [14]:
class MultiModal(nn.Module):
    def __init__(self, num_classes):
        super(MultiModal, self).__init__()
        self.cls = nn.Parameter(torch.randn(1, 1, 200))
        self.meta = Embedding(31,200)
        self.resnet18 = ResNet18(200)
        self.landsat = Embedding(504,200)
        self.position_landsat = nn.Parameter(torch.randn(1, 504))
        self.climate = Embedding(931,200)
        self.position_climate = nn.Parameter(torch.randn(1, 931))
        self.emb = Embedding(800,200)
        self.position_combine = nn.Parameter(torch.randn(1, 800))
        self.vit = ViT(200, 2, 200, 400, num_classes)
        self.position = nn.Parameter(torch.randn(1, 6, 200))

    def forward(self, x):
        batch = x[0].size(0)
        CLS = repeat(self.cls, '1 1 d -> b 1 d', b=batch).to(device)
        META = self.meta(x[0])
        IMG = self.resnet18(x[1])
        LANDSAT = self.landsat(x[2]+self.position_landsat)
        CLIMATE = self.climate(x[3]+self.position_climate)
        combine = torch.cat((META, IMG, LANDSAT, CLIMATE), dim=1)
        COMBINE = self.emb(combine+self.position_combine)
        token = torch.concat((CLS, META.unsqueeze(1)), dim=1)
        token = torch.concat((token, IMG.unsqueeze(1)), dim=1)
        token = torch.concat((token, LANDSAT.unsqueeze(1)), dim=1)
        token = torch.concat((token, CLIMATE.unsqueeze(1)), dim=1)
        token = torch.concat((token, COMBINE.unsqueeze(1)), dim=1)
        out = self.vit(token+self.position)
        return out

In [15]:
# Check if cuda is available
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("DEVICE = CUDA")

model = MultiModal(num_classes).to(device)
# model.load_state_dict(torch.load("models/vit/Model.pth", map_location=device))

DEVICE = CUDA


## Train

In [16]:
from torch.optim.lr_scheduler import CosineAnnealingLR
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00025)
# logger.info("Optimizer: AdamW")
scheduler = CosineAnnealingLR(optimizer, T_max=25, verbose=True)
# logger.info("Scheduler: CosineAnnealingLR")

/home/startung/miniconda3/envs/pt/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [17]:
print(f"Training for {num_epochs} epochs started.")

for epoch in tqdm(range(num_epochs)):
    model.train()

    for batch_idx, (sample, survey_id, labels, count) in enumerate(train_loader):
        samples = [tensor.to(device) for tensor in sample]
        survey_id = survey_id.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # with torch.set_grad_enabled(True):
        outputs = model(samples)

        pos_weight = labels*1.0  # All positive weights are equal to 10
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if batch_idx % (len(train_loader)//10) == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

    if epoch % 10 == 0:
        torch.save(model.state_dict(), new_dir / f"multimodal-epoch-{epoch}.pth")

    scheduler.step()
torch.save(model.state_dict(), new_dir / "multimodal-final.pth")

Training for 100 epochs started.


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100, Batch 0/1391, Loss: 0.7349907159805298
Epoch 1/100, Batch 139/1391, Loss: 0.029572946950793266
Epoch 1/100, Batch 278/1391, Loss: 0.014072494581341743
Epoch 1/100, Batch 417/1391, Loss: 0.010163888335227966
Epoch 1/100, Batch 556/1391, Loss: 0.00869766902178526
Epoch 1/100, Batch 695/1391, Loss: 0.007862644270062447
Epoch 1/100, Batch 834/1391, Loss: 0.008331937715411186
Epoch 1/100, Batch 973/1391, Loss: 0.007483727764338255
Epoch 1/100, Batch 1112/1391, Loss: 0.006611417513340712
Epoch 1/100, Batch 1251/1391, Loss: 0.006771528162062168


  1%|          | 1/100 [04:20<7:09:55, 260.56s/it]

Epoch 1/100, Batch 1390/1391, Loss: 0.005203625187277794
Epoch 2/100, Batch 0/1391, Loss: 0.005556628108024597
Epoch 2/100, Batch 139/1391, Loss: 0.005286202300339937
Epoch 2/100, Batch 278/1391, Loss: 0.0059546516276896
Epoch 2/100, Batch 417/1391, Loss: 0.00588897755369544
Epoch 2/100, Batch 556/1391, Loss: 0.005632961168885231
Epoch 2/100, Batch 695/1391, Loss: 0.005461188033223152
Epoch 2/100, Batch 834/1391, Loss: 0.0060033234767615795
Epoch 2/100, Batch 973/1391, Loss: 0.005516052711755037
Epoch 2/100, Batch 1112/1391, Loss: 0.0049687596037983894
Epoch 2/100, Batch 1251/1391, Loss: 0.005389644298702478


...

Epoch 16/100, Batch 1390/1391, Loss: 0.002798818051815033
Epoch 17/100, Batch 0/1391, Loss: 0.0031689044553786516
Epoch 17/100, Batch 139/1391, Loss: 0.0032997471280395985
Epoch 17/100, Batch 278/1391, Loss: 0.003566885832697153
Epoch 17/100, Batch 417/1391, Loss: 0.003676369786262512
Epoch 17/100, Batch 556/1391, Loss: 0.003653602907434106
Epoch 17/100, Batch 695/1391, Loss: 0.0035631456412374973
Epoch 17/100, Batch 834/1391, Loss: 0.003972898703068495
Epoch 17/100, Batch 973/1391, Loss: 0.0037647930439561605
Epoch 17/100, Batch 1112/1391, Loss: 0.0034953244030475616
Epoch 17/100, Batch 1251/1391, Loss: 0.003876172238960862


 17%|█▋        | 17/100 [1:14:21<6:03:15, 262.59s/it]

Epoch 17/100, Batch 1390/1391, Loss: 0.002753953682258725
Epoch 18/100, Batch 0/1391, Loss: 0.00313375610858202
Epoch 18/100, Batch 139/1391, Loss: 0.003273540874943137
Epoch 18/100, Batch 278/1391, Loss: 0.00353226438164711
Epoch 18/100, Batch 417/1391, Loss: 0.003660206450149417
Epoch 18/100, Batch 556/1391, Loss: 0.003602575743570924
Epoch 18/100, Batch 695/1391, Loss: 0.0035355279687792063
Epoch 18/100, Batch 834/1391, Loss: 0.00394072663038969
Epoch 18/100, Batch 973/1391, Loss: 0.0037234097253531218
Epoch 18/100, Batch 1112/1391, Loss: 0.0034835324622690678
Epoch 18/100, Batch 1251/1391, Loss: 0.0038301178719848394


 18%|█▊        | 18/100 [1:18:44<5:58:53, 262.61s/it]

Epoch 18/100, Batch 1390/1391, Loss: 0.002724003279581666
Epoch 19/100, Batch 0/1391, Loss: 0.0030956612899899483
Epoch 19/100, Batch 139/1391, Loss: 0.0032647359184920788
Epoch 19/100, Batch 278/1391, Loss: 0.0035039838403463364
Epoch 19/100, Batch 417/1391, Loss: 0.0036346372216939926
Epoch 19/100, Batch 556/1391, Loss: 0.0035827034153044224
Epoch 19/100, Batch 695/1391, Loss: 0.0035240361467003822


## Produce Submission

In [ ]:
with torch.no_grad():
    surveys = []
    top_indices = []
    for data, surveyID in tqdm(test_loader, total=len(test_loader)):

        data = [tensor.to(device) for tensor in data]

        outputs = model(data)
        predictions = torch.sigmoid(outputs).cpu().numpy()
        predictions = np.squeeze(predictions)
        # print(predictions)
        prediction = np.argwhere(predictions>=0.25).flatten()
        top_indices.append(prediction)
        surveys.extend(surveyID.cpu().numpy())

100%|██████████| 4716/4716 [00:11<00:00, 419.52it/s]


In [ ]:
data_concatenated = [' '.join(map(str, row)) for row in top_indices]

pd.DataFrame(
    {'surveyId': surveys,
     'predictions': data_concatenated,
    }).to_csv(new_dir / "submission.csv", index = False)

In [ ]:
total = 0
for row in top_indices:
    total += row.shape[0]

print(f"Total number of predictions: {total} with average of {total/len(top_indices)} predictions per survey ID.")

Total number of predictions: 79292 with average of 16.81340118744699 predictions per survey ID.
